**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Mar  9 08:31:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'17CS3651C Stat with R  lab Mannual (1).gdoc'
'17CS3651C Stat with R  lab Mannual (2).gdoc'
'17CS3651C Stat with R  lab Mannual (3).gdoc'
'17CS3651C Stat with R  lab Mannual (4).gdoc'
'17CS3651C Stat with R  lab Mannual.gdoc'
'17CS3651C Stat with R  lab Mannual.pdf'
'180_14_267807_1650292437_BPU Course Completion certificate.pdf'
'198W1A0538 (1).pdf'
'198W1A0538 Lab 1.gdoc'
'198W1A0538 Lab 1.pdf'
'198W1A0538 Lab 2.docx'
'198W1A0538 Lab 2.pdf'
'198W1A0538 Lab 3.docx'
'198W1A0538 Lab 3.pdf'
'198W1A0538 Lab 4.docx'
'198W1A0538 Lab 4.pdf'
 198w1a0538.mp4
 198W1A0538.pdf
'198W1A0538 SRAVANI MULLU.pdf'
 198W1A0546.jpeg
 198W1A1206-HA-1.docx
 198W1A1206-HA-2.docx
 198W1A1206-HA-3.docx
 2104.14506.pdf
'3.DMS PDNF & PCNF (1).gdoc'
'3.DMS PDNF & PCNF.gdoc'
'5. Methods of Proofs (1).gdoc'
'5. Methods of Proofs (2).gdoc'
'5. Methods of Proofs.gdoc'
'6. Predicate logic.gdoc'
'Application Layer.gslides'
 Classroom
'Colab Notebooks'
'Explainable_AI_in_Deep_Reinforcement_Lea

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.17 MiB | 14.98 MiB/s, done.
Resolving deltas: 100% (10412/10412), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "bridge" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-03-09 08:34:30--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  48.0MB/s    in 3.5s    

2023-03-09 08:34:34 (44.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/13.jpg         
  inflating: data/obj/13.txt         
  inflating: data/obj/14.jpg         
  inflating: data/obj/14.txt         
  inflating: data/obj/15.jpg         
  inflating: data/obj/15.txt         
  inflating: data/obj/16.jpg         
  inflating: data/obj/16.txt         
  inflating: data/obj/17.jpg         
  inflating: data/obj/17.txt         
  inflating: data/obj/18.jpg         
  inflating: data/obj/18.txt         
  inflating: data/obj/19.jpg         
  inflating: data/obj/19.txt         
  inflating: data/obj/20.jpg         
  inflating: data/obj/20.txt         
  inflating: data/obj/21.jpg         
  inflating: data/obj/21.txt         
  inflating: data/obj/22.jpg         
  inflating: data/obj/22.txt         
  inflating: data/obj/23.jpg         
  inflating: data/obj/23.txt         
  inflating: data/obj/24.jpg         
  inflating: data/obj/24.txt         
  inflating: data/obj/25.jpg         
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"/content/darknet/data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.298144 0.434156 0.238979 0.543210

['0', '0.298144', '0.434156', '0.238979', '0.543210']
0 /content/darknet/data/obj/40.txt
0 0.298144 0.434156 0.238979 0.543210
0 0.538283 0.456790 0.320186 0.629630

['0', '0.538283', '0.456790', '0.320186', '0.629630']
0 /content/darknet/data/obj/40.txt
0 0.538283 0.456790 0.320186 0.629630
0 0.284455 0.651408 0.152244 0.180751

['0', '0.284455', '0.651408', '0.152244', '0.180751']
1 /content/darknet/data/obj/92.txt
0 0.284455 0.651408 0.152244 0.180751
0 0.419872 0.489437 0.141026 0.157277

['0', '0.419872', '0.489437', '0.141026', '0.157277']
1 /content/darknet/data/obj/92.txt
0 0.419872 0.489437 0.141026 0.157277
0 0.703526 0.181925 0.080128 0.124413

['0', '0.703526', '0.181925', '0.080128', '0.124413']
1 /content/darknet/data/obj/92.txt
0 0.703526 0.181925 0.080128 0.124413
0 0.486207 0.387143 0.714943 0.540000

['0', '0.486207', '0.387143', '0.714943', '0.540000']
2 /content/darknet/data/obj/28.txt
0 0.486207 0.387143 0.714943 0.540000
0 0.

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/24.jpg', 'data/obj/63.jpg', 'data/obj/80.jpg', 'data/obj/17.jpg', 'data/obj/57.jpg', 'data/obj/78.jpg', 'data/obj/31.jpg', 'data/obj/47.jpg', 'data/obj/5.jpg', 'data/obj/44.jpg', 'data/obj/53.jpg', 'data/obj/70.jpg', 'data/obj/96.jpg', 'data/obj/90.jpg', 'data/obj/99.jpg', 'data/obj/94.jpg', 'data/obj/13.jpg', 'data/obj/45.jpg', 'data/obj/87.jpg', 'data/obj/19.jpg', 'data/obj/36.jpg', 'data/obj/83.jpg', 'data/obj/64.jpg', 'data/obj/9.jpg', 'data/obj/37.jpg', 'data/obj/71.jpg', 'data/obj/59.jpg', 'data/obj/46.jpg', 'data/obj/97.jpg', 'data/obj/61.jpg', 'data/obj/42.jpg', 'data/obj/88.jpg', 'data/obj/100.jpg', 'data/obj/38.jpg', 'data/obj/29.jpg', 'data/obj/43.jpg', 'data/obj/51.jpg', 'data/obj/3.jpg', 'data/obj/56.jpg', 'data/obj/93.jpg', 'data/obj/32.jpg', 'data/obj/11.jpg', 'data/obj/69.jpg', 'data/obj/7.jpg', 'data/obj/86.jpg', 'data/obj/85.jpg', 'data/obj/14.jpg', 'data/obj/50.jpg', 'data/obj/28.jpg', 'data/obj/55.jpg', 'data/obj/95.jpg', 'data/obj/74.jpg', 'data/obj/21.j

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 95317, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.722973), count: 4, class_loss = 0.184628, iou_loss = 0.242708, total_loss = 0.427336 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000076, iou_loss = 0.000000, total_loss = 0.000076 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 95321, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.734792), count: 5, class_loss = 0.332364, iou_loss = 0.284063, total_loss = 0.616427 
v3 (mse